# CROP BTS model
This notebook checks outputs of the BTS model

In [ ]:
#!pip3 install psycopg2
#!pip3 install plotly

In [ ]:
import os
from datetime import datetime, timedelta
import psycopg2
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt

Parameters

In [ ]:
crop_host = ""
crop_port = ""
crop_dbname = ""
crop_user = ""
crop_password = ""

In [ ]:
conn = psycopg2.connect(
    host=crop_host, port=crop_port, dbname=crop_dbname, 
    user=crop_user, password=crop_password)

cur = conn.cursor()

### BTS Data Access

All BTS data for 3 sensors

In [ ]:
#dt_to = datetime.now()
dt_to = datetime(2021, 4, 27)  
dt_from = dt_to + timedelta(days=-15)

In [ ]:
#sql_command = """SELECT * FROM model_run"""

sql_command = """SELECT model.id, model.model_name,  model_run.sensor_id, model_run.time_forecast, model_measure.measure_name, model_product.run_id, model_value.prediction_index,model_value.prediction_value 
FROM model, model_run, model_measure, model_product, model_value WHERE model.id =2 AND model_run.model_id = model.id  
AND model_product.run_id = model_run.id AND model_product.measure_id = model_measure.id AND model_value.product_id = model_product.id
AND model_run.time_forecast >= '%s' AND model_run.time_forecast < '%s'""" % (dt_from, dt_to)


In [ ]:
bts_raw = pd.read_sql(sql_command, conn)
bts = bts_raw.drop_duplicates()
bts

## Calculate and add column with prediciton time

In [ ]:
prediction_hours= []
for i in range(len(bts)):
   prediction_hours.append(bts['time_forecast'][i] + timedelta(hours= int(bts['prediction_index'][i])))

bts['prediction_time'] = prediction_hours

bts


In [ ]:
#test = bts[bts['run_id'] ==  86]


In [ ]:
#unique_run_ids = bts['run_id'].unique()
unique_sensors = bts['sensor_id'].unique()
unique_time_forecast = bts['time_forecast'].unique()
unique_measures = bts['measure_name'].unique()

unique_time_forecast

In [ ]:
date_time = pd.to_datetime(unique_time_forecast[0])
date_time
#type(date_time)
#type(datetime(2021, 6, 16))

### Get Temperature data from Zensie Sensors

All Zensie sensors

In [ ]:
dt_to_z = date_time + timedelta(days=+3) #datetime(2021, 6, 16)
dt_from_z = dt_to_z + timedelta(days=-6)

In [ ]:
sql_command_zensie = """SELECT sensors.name, zensie_trh_data.* FROM sensor_types, sensors, zensie_trh_data WHERE sensors.id = zensie_trh_data.sensor_id AND zensie_trh_data.timestamp >= '%s' AND zensie_trh_data.timestamp < '%s'""" % (dt_from_z, dt_to_z)

In [ ]:
zensie_raw = pd.read_sql(sql_command_zensie, conn)
zensie_df = zensie_raw.drop_duplicates()
zensie_df

## Visualisation and parameters

In [ ]:
time_forecast_id= 0
time_forecast = unique_time_forecast[time_forecast_id]

In [ ]:
color = ["red", 'blue', 'white']
for j in range (len(unique_sensors)):

    # set up plotly figure
    fig = go.Figure()
    
    bts_df = bts[(bts['sensor_id'] == unique_sensors[j]) & (bts['time_forecast'] == time_forecast)]

    # add line / trace 1 to figure
    for i in range(len (unique_measures)):
        trace_sensor = bts_df[bts_df['measure_name'] == unique_measures[i]]
        fig.add_trace(go.Scatter(
            x=trace_sensor['prediction_time'],
            y=trace_sensor['prediction_value'],
            #hovertext=df['A_info'],
            #hoverinfo="text",
            name=unique_measures[i],#unique_measures[j],
            marker=dict(
                color=color[i],
            ),
            showlegend=True
        ))

    # add line / trace 2 to figure
    
    zensie_df_s = zensie_df[zensie_df['sensor_id'] == unique_sensors[j]]
    fig.add_trace(go.Scatter(
        x=zensie_df_s['timestamp'],
        y=zensie_df_s['temperature'],
        name='zensie',
        marker=dict(
            color="green"
        ),
        showlegend=False
    ))
    
    fig.update_layout(
    title="BTS and Zensie data: Sensor id %s" % unique_sensors[j] ,
    xaxis_title="Date",
    yaxis_title="Temperature")

    fig.show()

## Calculate Square root mean error

In [ ]:
import sklearn
from sklearn.metrics import mean_squared_error

In [ ]:
unique_measures